In [2]:
#!pip install ./src

In [1]:
import sys
sys.path.append("/workspace/maskrcnn/pytorch/")
import argparse
import os
import functools
import logging
import random
import datetime
import time
import gc
import numpy as np

import torch
import apex_C, amp_C

from s3fs import S3FileSystem

In [2]:
# Get weights and annotations from S3
s3 = S3FileSystem()
anno_dir = "/opt/ml/input/data/annotations"
weights_dir = "/opt/ml/input/data/weights"
weights_file = "resnet50.pkl"
# os.makedirs(weights_dir, exist_ok=True)
# os.makedirs(anno_dir, exist_ok=True)
# s3.get("s3://jbsnyder-sagemaker-us-east/data/weights/resnet50.pkl", os.path.join(weights_dir, weights_file))
# s3.get("s3://jbsnyder-sagemaker-us-east/data/coco/annotations/", anno_dir, recursive=True)

In [3]:
from apex.multi_tensor_apply import multi_tensor_applier

In [4]:
from maskrcnn_benchmark.config import cfg
from src.build import make_data_loader
from maskrcnn_benchmark.data.datasets.coco import HybridDataLoader3
from maskrcnn_benchmark.solver import make_lr_scheduler
from maskrcnn_benchmark.solver import make_optimizer
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.engine.trainer import do_train
from maskrcnn_benchmark.engine.tester import test
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank, is_main_process, get_world_size, is_main_evaluation_process
from maskrcnn_benchmark.utils.imports import import_file
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir
from maskrcnn_benchmark.utils.mlperf_logger import log_end, log_start, log_event, generate_seeds, broadcast_seeds, barrier, configure_logger
from maskrcnn_benchmark.utils.async_evaluator import init, get_evaluator, set_epoch_tag, get_tag
from maskrcnn_benchmark.utils.timed_section import TimedSection

In [5]:
from fp16_optimizer import FP16_Optimizer
from apex import amp
from apex.parallel import DistributedDataParallel as DDP

In [6]:
num_gpus = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
distributed = num_gpus > 1

In [7]:
world_size = 1
rank = 0
# random master seed, random.SystemRandom() uses /dev/urandom on Unix
master_seed = random.SystemRandom().randint(0, 2 ** 32 - 1)
random_number_generator = random.Random(master_seed)

In [8]:
cfg.merge_from_file('src/configs/e2e_mask_rcnn_R_50_FPN_1x.yaml')
cfg.PATHS_CATALOG = "src/s3_paths_catalog.py"
# cfg.freeze()

In [9]:
cfg.MODEL.WEIGHT = os.path.join(weights_dir, weights_file)

In [10]:
model = build_detection_model(cfg)

:::MLLOG {"namespace": "", "time_ms": 1652237660216, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/workspace/maskrcnn/pytorch/maskrcnn_benchmark/modeling/backbone/fpn.py", "lineno": 44, "tensor": "FPN_inner_block1"}}
:::MLLOG {"namespace": "", "time_ms": 1652237660317, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/workspace/maskrcnn/pytorch/maskrcnn_benchmark/modeling/backbone/fpn.py", "lineno": 47, "tensor": "FPN_layer_block1"}}
:::MLLOG {"namespace": "", "time_ms": 1652237660320, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/workspace/maskrcnn/pytorch/maskrcnn_benchmark/modeling/backbone/fpn.py", "lineno": 44, "tensor": "FPN_inner_block2"}}
:::MLLOG {"namespace": "", "time_ms": 1652237660330, "event_type": "POINT_IN_TIME", "key": "weights_initialization", "value": null, "metadata": {"file": "/workspace/maskrcnn/pytorch

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/conv.py:1092: UserWarning: _ConvTransposeMixin is a deprecated internal class. Please consider using public APIs.
  warnings.warn(


In [11]:
device = torch.device(cfg.MODEL.DEVICE)

In [15]:
data_loader, iters_per_epoch = make_data_loader(
        cfg,
        is_train=True,
        is_distributed=distributed,
        start_iter=0,
        random_number_generator=random_number_generator,
        seed=master_seed,
        shapes=None,
        hybrid_dataloader=None,
    )

'data_loader, iters_per_epoch = make_data_loader(\n        cfg,\n        is_train=True,\n        is_distributed=distributed,\n        start_iter=0,\n        random_number_generator=random_number_generator,\n        seed=master_seed,\n        shapes=None,\n        hybrid_dataloader=None,\n    )'